# Loading Pre-Traing LLM model

In [1]:
import os
os.path.exists('../_results')

True

In [2]:
from transformers import BertForQuestionAnswering, BertTokenizerFast
import torch

# Load the trained model and tokenizer
model = BertForQuestionAnswering.from_pretrained(pretrained_model_name_or_path="D:\_results_7\checkpoint-75")
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path="D:\_results_7\checkpoint-75")

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [3]:
def predict_answer(question, context):
    # Tokenize the input question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
    # Get the start and end logits (the raw outputs)
    start_logits, end_logits = outputs.start_logits, outputs.end_logits

    # Get the most likely start and end positions
    start_pos = torch.argmax(start_logits, dim=1)
    end_pos = torch.argmax(end_logits, dim=1)

    # Convert the token positions to the actual answer text
    answer_tokens = input_ids[0][start_pos:end_pos+1]
    answer = tokenizer.decode(answer_tokens)

    return answer


In [4]:
# Example question and context
question = "I have a microscopic image that has ovel with a thick, bumpy shell. What parasitic egg is it?"
context = "The eggs of Ascaris lumbricoides, Capillaria philippinensis, Enterobius vermicularis, and Fasciolopsis buski have distinct microscopic features and cause varying symptoms. Ascaris lumbricoides eggs are oval with a thick, bumpy shell, and infections can lead to abdominal pain or respiratory symptoms, treatable with albendazole. Capillaria philippinensis eggs are peanut-shaped with flattened polar plugs, causing diarrhea and weight loss, also treated with albendazole. Enterobius vermicularis eggs are oval and clear, primarily causing perianal itching, treatable with mebendazole. Fasciolopsis buski eggs are large, oval, and have an operculum, leading to abdominal pain and diarrhea, with praziquantel as the treatment. Hygiene and proper sanitation are key to preventing these infections."

# Get the model's predicted answer
predicted_answer = predict_answer(question, context)

print(f"Question: {question}")
print(f"Context: {context}")
print(f"Predicted Answer: {predicted_answer}")


Question: I have a microscopic image that has ovel with a thick, bumpy shell. What parasitic egg is it?
Context: The eggs of Ascaris lumbricoides, Capillaria philippinensis, Enterobius vermicularis, and Fasciolopsis buski have distinct microscopic features and cause varying symptoms. Ascaris lumbricoides eggs are oval with a thick, bumpy shell, and infections can lead to abdominal pain or respiratory symptoms, treatable with albendazole. Capillaria philippinensis eggs are peanut-shaped with flattened polar plugs, causing diarrhea and weight loss, also treated with albendazole. Enterobius vermicularis eggs are oval and clear, primarily causing perianal itching, treatable with mebendazole. Fasciolopsis buski eggs are large, oval, and have an operculum, leading to abdominal pain and diarrhea, with praziquantel as the treatment. Hygiene and proper sanitation are key to preventing these infections.
Predicted Answer: 
